In [33]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("train.csv")


Look for the missing values in all the columns and either impute them (replace with mean, median, or mode) or drop them. Justify your action for this task.

In [34]:
# Helper function to extract numeric part from strings
def extract_numeric(series):
    return pd.to_numeric(series.str.extract(r'([\d.]+)')[0], errors='coerce')

# Clean and convert string columns with units to numeric
df['Mileage'] = extract_numeric(df['Mileage'])
df['Engine'] = extract_numeric(df['Engine'])
df['Power'] = extract_numeric(df['Power'])

# Impute missing values
df['Mileage'].fillna(df['Mileage'].median(), inplace=True)
df['Engine'].fillna(df['Engine'].median(), inplace=True)
df['Power'].fillna(df['Power'].median(), inplace=True)
df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)


# Final check for any remaining missing values
print("Total missing values:", df.isnull().sum().sum())

Total missing values: 5032


<ipython-input-34-3a7589b20ef4>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Mileage'].fillna(df['Mileage'].median(), inplace=True)
<ipython-input-34-3a7589b20ef4>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

b) Remove the units from some of the attributes and only keep the numerical values (for example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from “New_price”)

In [35]:
def extract_numeric(series):
    return series.astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float)

In [36]:
def extract_numeric(series):
    return series.apply(lambda x: float(str(x).split()[0]) if pd.notnull(x) and str(x).strip() != '' else None)

In [37]:
df['Mileage'] = extract_numeric(df['Mileage'])
df['Engine'] = extract_numeric(df['Engine'])
df['Power'] = extract_numeric(df['Power'])
df['New_Price'] = df['New_Price'].str.replace(' Lakh', '', regex=False)
df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce') * 100000  # Convert lakh to rupees



In [38]:
print(df.columns.tolist())


['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price']


C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.

In [39]:
# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# Check result
print(df.head())

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0              41000      First    19.67  1582.0  126.20    5.0        NaN   
1              46000      First    13.00  1199.0   88.70    5.0   861000.0   
2              87000      First    20.77  1248.0   88.76    7.0        NaN   
3              40670     Second    15.20  1968.0  140.80    5.0        NaN   
4              86999      First    23.08  1461.0   63.10    5.0        NaN   

   Price  Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Manual  
0  12.50               False             False

In [44]:
from datetime import datetime

# Get current year
current_year = datetime.now().year

# Add Car_Age to the cleaned dataframe
df['Car_Age'] = current_year - df['Year']

# Display result
print(df[['Car_Age']].head())

   Car_Age
0       10
1       14
2       13
3       12
4       12


E) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or
their equivalent operations in python) on this dataset.

In [46]:
# Select
df_selected = df[['Owner_Type', 'Price', 'Mileage']]

# Filter
df_filtered = df[df['Mileage'] > 25]

# Rename
df_renamed = df.rename(columns={'Owner_Type': 'Brand', 'Price': 'Car_Price'})

# Mutate
df['Power_kW'] = df['Power'] * 0.7457

# Arrange
df_sorted = df.sort_values(by='Price', ascending=False)

# Group by and summarize
df_summary = df.groupby('Owner_Type')['Mileage'].mean().reset_index()

df_summary_multi = df.groupby('Owner_Type').agg(
    total_price=('Price', 'sum'),
    avg_mileage=('Mileage', 'mean'),
    max_power=('Power', 'max')
).reset_index()

# Show final results
print("Selected Data:")
print(df_selected)
print("\nFiltered Data:")
print(df_filtered)
print("\nRenamed Data:")
print(df_renamed)
print("\nSorted Data:")
print(df_sorted)
print("\nGrouped and Summarized Data:")
print(df_summary_multi)

Selected Data:
     Owner_Type  Price  Mileage
0         First  12.50    19.67
1         First   4.50    13.00
2         First   6.00    20.77
3        Second  17.74    15.20
4         First   3.50    23.08
...         ...    ...      ...
5842      First   4.75    28.40
5843      First   4.00    24.40
5844     Second   2.90    14.00
5845      First   2.65    18.90
5846      First   2.50    25.44

[5847 rows x 3 columns]

Filtered Data:
      Unnamed: 0                                       Name    Location  Year  \
10            12                      Maruti Swift VDI BSIV      Jaipur  2015   
14            16                      Honda Amaze S i-Dtech       Kochi  2016   
15            17                      Maruti Swift DDiS VDI      Jaipur  2017   
39            41                            Tata Nano LX SE       Kochi  2014   
40            42  Mahindra KUV 100 mFALCON D75 K8 Dual Tone  Coimbatore  2017   
...          ...                                        ...         ...   